# Detecting floating objects using deep learning and Sentinel-2 imagery

:::{eval-rst}
:opticon:`tag`
:badge:`Ocean,badge-primary`
:badge:`Modelling,badge-secondary`
:::

## Context

### Purpose
Detect floating targets that might contain plastic, algea, sargassum, wood, etc. in coastal areas using deep learning methods and Sentinel-2 data.

### Modelling approach

In this notebook we show the potential of deep learning methods to detect and highlight floating debris of various natures in coastal areas. Initially, we demonstrate how to download a Sentinel-2 image over a coastal area of interest directly from this notebook. We also provide the possibility to load an image provided with this notebook in case the user does not wish to do the downloading. Once the image is downloaded/loaded, we apply pre-trained weights in order to identify the potential existence of floating targets. Our experiments were implemented using Pytorch. Further details on the labeled dataset and the code can be found here: https://github.com/ESA-PhiLab/floatingobjects.

### Highlights

* We demonstrate the use of deep neural networks for the detection of floating objects on Sentinel-2 data.
* We give the user the possibility to specify the coordinates of a coastal Area Of Interest (AOI) of their choice in the reference system WGS84 (EPSG:4326).
* The exact AOI can be displayed on a real map to verify the coordinates.
* The user can then directly download the corresponding Sentinel-2 image, if available, and apply the pre-trained weights. Alternatively, the user can just load the image provided with this notebook to run the predictions.
* Other use cases are included in order to show the variety of the detected objects.
* Finally, the user can visualise the RGB image, the NDVI and FDI indices along with the predictions and classifications.
* Remark: if the user wants to apply the model on a specific '.tif' image, it's possible to skip ahead to the **Compute model predictions** section.

### Contributions

#### Notebook
* Jamila Mifdal (author), European Space Agency Φ-lab, [@jmifdal](https://github.com/jmifdal)
* Raquel Carmo (author), European Space Agency Φ-lab, [@raquelcarmo](https://github.com/raquelcarmo)
* Alejandro Coca-Castro (reviewer), The Alan Turing Institute, [@acocac](https://github.com/acocac)

#### Modelling codebase
* Jamila Mifdal (author), European Space Agency Φ-lab, [@jmifdal](https://github.com/jmifdal)
* Raquel Carmo (author), European Space Agency Φ-lab, [@raquelcarmo](https://github.com/raquelcarmo)
* Marc Rußwurm (author), EPFL-ECEO, [@marccoru](https://github.com/MarcCoru)

#### Modelling publications
```{bibliography}
  :style: plain
  :list: bullet
  :filter: topic % "ocean-modelling-floatingobjects_philab"
```

#### Modelling funding
TBC

:::{note}
The notebook contributors acknowledge ...
:::

## Install and load libraries

In [ ]:
#Install sentinelsal library
!pip -q install sentinelsat --upgrade

#Install pytorch and segmentation models
!pip -q install torch
!pip -q install segmentation_models_pytorch

#Install geospatial libraries
!pip -q install geopandas
!pip -q install fiona
!pip -q install rasterio
!pip -q install shapely

#Install library to retieve data from zenodo
!pip3 install zenodo_get
#Alternatively run: !pip3 install git+https://gitlab.com/dvolgyes/zenodo_get

#Install interactive plotting
!pip -q install holoviews
!pip -q install hvplot

#Install other libraries
!pip -q install folium natsort xarray scikit-image xmlschema lxml gdown

The data to be used in this notebook can be fetched directly from a [Zenodo repository](https://zenodo.org/record/5827377#.YdgfjGjMK9I) using the cell below. We provide the repository DOI (10.5281/zenodo.5827377) and we set the output folder as `./ocean-modelling-litter-philab/`, where the downloaded files will be stored. Since the zenodo repository is relatively heavy (~1GB), the download might take some time.

In [ ]:
#Download the zenodo repository where the files used in this notebook are available
!zenodo_get 10.5281/zenodo.5827377 --output-dir=./ocean-modelling-litter-philab/

In [ ]:
import numpy as np
import os, json
from glob import glob
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from natsort import natsorted
import xml.etree.ElementTree as ET
import xarray as xr
import shutil
import gdown

#Machine learning libraries
import torch
import segmentation_models_pytorch

#Visualisation
import matplotlib, matplotlib.cm, param
import matplotlib.pyplot as plt
from skimage.exposure import equalize_hist
import holoviews as hv
from holoviews import opts
import panel as pn
import hvplot
import hvplot.xarray  # noqa

#Geospatial libraries
from shapely.geometry import box
import geopandas as gpd
import folium
from fiona.crs import from_epsg
import rasterio as rio
from rasterio.mask import mask
from rasterio.windows import Window
from rasterio.plot import show

import warnings
warnings.filterwarnings('ignore')

## Set folder structure

In [ ]:
# Define the project main folder
data_folder = './ocean-modelling-litter-philab'

# Set the folder structure
config = {
    'in_geotiff': os.path.join(data_folder, 'input','tiff'),
    'in_geojson': os.path.join(data_folder, 'input','geojson'),
    'out_geotiff': os.path.join(data_folder, 'output','raster'),
}

# List comprehension for the folder structure code
[os.makedirs(val) for key, val in config.items() if not os.path.exists(val)]

## Fetch a Sentinel-2 image of a coastal area
In this section, we suggest the possibility to download a Sentinel-2 image of an area of interest (AOI) using the [sentinelhub API](https://pythonrepo.com/repo/sentinelsat-sentinelsat-python-geolocation). In order to download the data, the user has to register following this link: https://scihub.copernicus.eu/userguide/SelfRegistration. 

After registering, the user can provide a GeoJSON file of an AOI using for example this website: http://geojson.io/. In the example below, an area of Rio De Janeiro in Brazil was selected using a polygon and the corresponding GeoJSON file 'RioDeJaneiro.geojson' was dowloaded and provided for the Sentinel-2 image search.

The 'api.query' will provide the available products corresponding to the search keywords. It is easier to download a product when it's online, to verify this, 'api.get_product_odata' is applied given the product's uuid. When the product is not online it means that it's in the LTA (Long Term Archive). Recovering data from the LTA is not covered in the example below.

*Remark*: If the product that we suggest to download is no longer available online, please include the id of an online product or simply use the provided .zip file starting with 'S2' that can be retrieved through the Zenodo repository.

### Sentinel-2 product search

In [ ]:
#Connect to the API by changing 'user' and 'password' accordingly
api = SentinelAPI('user', 'password', 'https://scihub.copernicus.eu/dhus')

#Search by polygon, time and Hub query keywords
footprint = geojson_to_wkt(read_geojson('./ocean-modelling-litter-philab/RioDeJaneiro.geojson'))

#Query the API for the products
products = api.query(footprint,
                     date = ('20210730','20210830'),
                     platformname = 'Sentinel-2',
                     cloudcoverpercentage = (0, 30))

#Display the products and their ids ('uuid')
products

In [ ]:
#Determine the product-of-interest's id and check the availability (the field 'online' should be 'True')
info_product = api.get_product_odata('08ec0d95-6a93-493f-b801-7bd11b6c50bd')

#Download the product of interest by entering the corresponging id
api.download('08ec0d95-6a93-493f-b801-7bd11b6c50bd')

### Read and sort the bands

In [ ]:
#Extract the downloaded .zip file
shutil.unpack_archive(info_product['title']+'.zip', './')

#List the bands corresponding to each spatial resolution
list_bands_10m = glob(os.path.join(info_product['title']+'.SAFE/GRANULE/','*','*','R10m','*_B*.jp2'))
list_bands_20m = glob(os.path.join(info_product['title']+'.SAFE/GRANULE/','*','*','R20m','*_B*.jp2'))
list_bands_60m = glob(os.path.join(info_product['title']+'.SAFE/GRANULE/','*','*','R60m','*_B*.jp2'))

def find_bands(wanted_strings, list_names):
    new_list = []
    for f in list_names:
        #Use replace() to ensure operability between os
        s = f.replace('\\','/').split('/')[-1].split('_')[2]
        for w in wanted_strings:
            if s == w:
                new_list.append(f.replace('\\','/'))
    return new_list

#Get the bands corresponding to the right resolutions of Sentinel-2 products
bands_10m = find_bands(['B02','B03','B04','B08'], list_bands_10m)
bands_20m = find_bands(['B05','B06','B07','B8A','B11','B12'], list_bands_20m)
bands_60m = find_bands(['B01','B09','B10'], list_bands_60m)

listAllJP2 = bands_10m + bands_20m + bands_60m

#Sort the bands
bandsSorted = natsorted(listAllJP2, key = lambda x: x.split('/')[-1].split('_')[2])
bandsSorted

Here we display the bands in an interactive dashboard.

In [ ]:
def read_band(path):
    '''Read each band from path'''
    with rio.open(path) as src_b:
        b1 = src_b.read()
        b = np.squeeze(b1, axis=0)
    return b

def hook(plot, element):
    '''Remove axis labels'''
    plot.state.xaxis.axis_label = None
    plot.state.yaxis.axis_label = None

bands = [i.split('/')[-1].split('_')[2] for i in bandsSorted]
images = list(map(read_band, bandsSorted))

#Class to display dashboard
class BandDashboard(param.Parameterized):
    band_selected = param.Selector(
        objects=bands,
        default=bands[0],
        label='Select band'
    )
    
    @param.depends('band_selected')
    def plot(self):
        index = [ind for ind, path in enumerate(bandsSorted) if self.band_selected in path][0]
        
        img = images[index]
        h,w = img.shape
        bounds = (0,0,w,h)
        return hv.Image(img, bounds=bounds)\
                 .opts(title=f'Band {self.band_selected}', hooks=[hook], cmap='terrain',
                       colorbar=True, tools=['hover'], height=400, width=500)
    
    def panel(self):
        return pn.Row(self.param, self.plot)

dashboard = BandDashboard(name='Band Visualization')
dashboard.panel()

### Get the geo-information data

In [ ]:
#Get the coordinates (WGS84) from the geojson file and set them as a bounding box (bbox)
geojson_info = read_geojson('./ocean-modelling-litter-philab/RioDeJaneiro.geojson')
coords_WGS84 = geojson_info["features"][0]["geometry"]["coordinates"]
bbox = box(coords_WGS84[0][0][0], coords_WGS84[0][0][1], coords_WGS84[0][2][0], coords_WGS84[0][2][1]) 


#Get the 'crs' info in the xml file downloaded with the data 
xml_path = glob(os.path.join(info_product['title']+'.SAFE/GRANULE/','*','MTD_TL.xml'))
tree = ET.parse(xml_path[0])
root = tree.getroot()
crs_json = int(root[1][0][1].text.split(':')[1])


#Insert the bbox into a GeoDataFrame
geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(4326)) 

#Re-project into the same coordinate system as the raster data 
geo = geo.to_crs(crs_json) #Rio de janeiro

#Function to parse features from GeoDataFrame in such a manner that rasterio wants them
def getFeatures(gdf):
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

features = getFeatures(geo)
features[0]

### Crop the bands to the exact AOI

In [ ]:
#Get the coordinates of the cropping window
coords = features[0]
c_left = coords['coordinates'][0][2][0]
c_top = coords['coordinates'][0][2][1]
c_right = coords['coordinates'][0][0][0]
c_bottom = coords['coordinates'][0][0][1]

#Get a window from the coordinates defined above and a transform of a high resolution band displayed above
with rio.open(bandsSorted[1]) as src_b:
    b1 = src_b.read()
    b1 = np.squeeze(b1, axis=0)
    window2crop = rio.windows.from_bounds(
        left=c_left,
        bottom=c_bottom,
        right=c_right,
        top=c_top,
        transform=src_b.transform
    )

#Stack the cropped bands in one tif image using the high resolution band dimensions
with rio.open('riodejaneiro.tif','w', driver='JP2OpenJPEG', 
              height=window2crop.height, width=window2crop.width, 
              count=13, dtype = 'uint16', transform=src_b.transform) as dst:
        
    for id, b in enumerate(bandsSorted, start=1):
        with rio.open(b) as src:
            
            _, outTransform = mask(src, features, crop=False)
    
            window2crop = rio.windows.from_bounds(
                left=c_left,
                bottom=c_bottom,
                right=c_right,
                top=c_top,
                transform=outTransform
            )
            outImage = src.read(1, window=window2crop)

            #Save the image locally as 'riodejaneiro.tif'
            dst.write_band(id, outImage)

#Read the reconstructed image saved locally
with rio.open('./riodejaneiro.tif','r') as src_pred:
    meta = src_pred.meta
    image = src_pred.read()

Here we test the cropping procedure using band B02 for visualization purposes.

In [ ]:
#Display a 10m-resolution band (B02) which is the downloaded scene
B02 = rio.open(bandsSorted[1]).read()

#Convert to 'xarray.DataArray'
B02_xr = xr.DataArray(np.squeeze(B02, axis=0), dims=['y', 'x'], 
                      coords={'y': np.arange(B02.shape[1]), 
                              'x': np.arange(B02.shape[2])})

B02_xr.hvplot(x='x', y='y', data_aspect=1, flip_yaxis=True, xaxis=False, yaxis=None, cmap='terrain', title = 'B02 (10m)')

### Display the exact AOI on a real map 

In [ ]:
#Insert the bbox into a GeoDataFrame
geo_map = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(4326))
center = geo_map.centroid
lat, lon = center.y[0], center.x[0]
m = folium.Map(location=[lat,lon], zoom_start=11, max_bounds=False)
folium.Marker(location=[lat,lon]).add_to(m)
folium.GeoJson(geo_map).add_to(m)
m

## Load Sentinel-2 data from a local source (.tif format)
Alternatively to the procedure above, the user could also load a Sentinel-2 image available locally. It would be preferable if the user could provide the bounding box of the S2 scene using the WGS84 coordinates that could easily be recovered, for instance, from Google Earth Engine.

In the example below, we load the .tif image provided in the [Zenodo repository](https://zenodo.org/record/5827377#.YdgfjGjMK9I), called "RioDeJaneiro.tif".

In [ ]:
#Load the .tif image
path_image = './ocean-modelling-litter-philab/RioDeJaneiro.tif'

with rio.open(path_image, "r") as src_pred:
    meta = src_pred.meta
    image = src_pred.read()    
image.shape

## Compute model predictions
In this work we focus on the spatial patterns of the floating targets, thus we address this detection task as a binary classification problem of floating objects versus water surface. A deep learning-based segmentation model was chosen to perform the detection and delineation of the floating targets: a U-Net Convolutional Neural Network (CNN). This model has been pre-trained on a large open-source hand-labeled Sentinel-2 dataset over coastal water bodies, developed by the authors. For generalization purposes, this dataset includes Sentinel-2 products from both Level-1C Top-Of-Atmosphere (TOA) and Level-2A Bottom-Of-Atmosphere (BOA). Please refer to the **Modelling publications** section for further details.

### Load the model and its pre-trained weights
For the U-Net, there are two pre-trained models available (described in this [link](https://github.com/ESA-PhiLab/floatingobjects/blob/master/hubconf.py)). For the sake of simplicity we are loading the weights of the 'unet_seed0', which refers to a U-Net architecture pre-trained for 50 epochs, with batch size of 160, learning rate of 0.001 and seed 0.

In [ ]:
#Device to run the computations on
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#Available models and weights
unet_seed0 = torch.hub.load('ESA-PhiLab/floatingobjects:master', 'unet_seed0', map_location=torch.device(device))
#unet_seed1 = torch.hub.load('ESA-PhiLab/floatingobjects:master', 'unet_seed1', map_location=torch.device(device))

#Select model
model = unet_seed0

### Compute predictions
Input the image to the model to yield predictions.

In [ ]:
#Predict the floating objects with the model specified above
l1cbands = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B10", "B11", "B12"]
l2abands = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B11", "B12"]

#If L1C image (13 bands), read only the 12 bands compatible with L2A data
if (image.shape[0] == 13):
    image = image[[l1cbands.index(b) for b in l2abands]]
image = image.astype(float)
image *= 1e-4
image = torch.from_numpy(image)

#Compute predictions
with torch.no_grad():
    x = image.unsqueeze(0)
    y_logits = torch.sigmoid(model(x.to(device)).squeeze(0))
    y_score = y_logits.cpu().detach().numpy()[0]

### Compute NDVI and FDI indices
For detecting marine litter, pixelwise spectral features such as the Normalized Difference Vegetation Index (NDVI) and the [Floating Debris Index (FDI)](https://www.nature.com/articles/s41598-020-62298-z) are often chosen as problem-specific features when using model-driven classifiers (e.g. Random Forest or Naïve Bayes). In this case, because we're applying a data-driven approach (U-Net), we only resort to these indices for visual inspection purposes.
To compute these indices, we used the following equations:

$$NDVI=\dfrac{R_{rs,NIR}-R_{rs,RED}}{R_{rs,NIR}+R_{rs,RED}}$$

$$FDI=R_{rs,NIR}-R'_{rs,NIR}$$

$$R'_{rs,NIR} = R_{rs,RED_2} + (R_{rs,SWIR_1} - R_{rs,RED_2}) \times \dfrac{(\lambda_{NIR} - \lambda_{RED})}{(\lambda_{SWIR_1} - \lambda_{RED})} \times 10$$

where:

- $R_{rs,NIR}$ is the spectral reflectance measured in the near infrared waveband (band B08)
- $R_{rs,RED}$ is the spectral reflectance measured in the red waveband (band B04)
- $R_{rs,RED_2}$ is the spectral reflectance measured in the red edge waveband (band B06)
- $R_{rs,SWIR_1}$ is the spectral reflectance measured in the shortwave infrared (band B11)
- $\lambda_{NIR} = 832.9$
- $\lambda_{RED} = 664.8$
- $\lambda_{SWIR_1} = 1612.05$

In [ ]:
def calculate_fdi(scene):
    '''Compute FDI index'''
    NIR = scene[l2abands.index("B8")] 
    RED2 = scene[l2abands.index("B6")]
    SWIR1 = scene[l2abands.index("B11")] 

    lambda_NIR = 832.9
    lambda_RED = 664.8
    lambda_SWIR1 = 1612.05
    NIR_prime = RED2 + (SWIR1 - RED2) * 10 * (lambda_NIR - lambda_RED) / (lambda_SWIR1 - lambda_RED)
    return NIR - NIR_prime

def calculate_ndvi(scene):
    '''Compute NDVI index'''
    NIR = scene[l2abands.index("B8")].float()
    RED = scene[l2abands.index("B4")].float()
    return (NIR - RED) / (NIR + RED + 1e-12)

In [ ]:
#Compute the NDVI and FDI bands corresponding to the image
fdi = calculate_fdi(image).cpu().detach().numpy()
fdi = np.expand_dims(fdi,0)
fdi = np.squeeze(fdi,0)
ndvi = calculate_ndvi(image).cpu().detach().numpy()

#Compute RGB representation
tensor = np.stack([image[l2abands.index('B4')], image[l2abands.index('B3')], image[l2abands.index('B2')]])
rgb = equalize_hist(tensor.swapaxes(0,1).swapaxes(1,2))

#Configure visualisation settings
cmap_magma = matplotlib.cm.get_cmap('magma')
cmap_viridis = matplotlib.cm.get_cmap('viridis')
cmap_terrain = matplotlib.cm.get_cmap('terrain')
norm_fdi = matplotlib.colors.Normalize(vmin=0, vmax=0.1)
norm_ndvi = matplotlib.colors.Normalize(vmin=-.4, vmax=0.4)
norm = matplotlib.colors.Normalize(vmin=0, vmax=0.4)

Here we create the interactive plots.

In [ ]:
general_settings = {'x':'x', 'y':'y', 'data_aspect':1, 'flip_yaxis':True, 
                    'xaxis':False, 'yaxis':None, 'tools':['tap', 'box_select']}

#RGB
#convert to 'xarray.DataArray'
RGB_xr = xr.DataArray(rgb, dims=['y', 'x', 'band'], 
                      coords={'y': np.arange(rgb.shape[0]),
                              'x': np.arange(rgb.shape[1]), 
                              'band': np.arange(rgb.shape[2])})
plot_RGB = RGB_xr.hvplot.rgb(**general_settings, bands='band', title='RGB')

#FDI
FDI = cmap_magma(norm_fdi(fdi))
FDI_tmp = FDI[:,:,0:3]
#convert to 'xarray.DataArray'
FDI_xr = xr.DataArray(FDI_tmp, dims=['y', 'x', 'band'], 
                      coords={'y': np.arange(FDI_tmp.shape[0]),
                              'x': np.arange(FDI_tmp.shape[1]), 
                              'band': np.arange(FDI_tmp.shape[2])})
plot_FDI = FDI_xr.hvplot.rgb(**general_settings, bands='band', title='FDI')

#NDVI
NDVI = cmap_viridis(norm_ndvi(ndvi))
NDVI_tmp = NDVI[:,:,0:3]
#convert to 'xarray.DataArray'
NDVI_xr = xr.DataArray(NDVI_tmp, dims=['y', 'x', 'band'], 
                       coords={'y': np.arange(NDVI_tmp.shape[0]),
                               'x': np.arange(NDVI_tmp.shape[1]),
                               'band': np.arange(NDVI_tmp.shape[2])})
plot_NDVI = NDVI_xr.hvplot.rgb(**general_settings, bands='band', title='NDVI')

#Predictions
Predictions = cmap_magma(norm(y_score))
#convert to 'xarray.DataArray'
Predictions_xr = xr.DataArray(Predictions, dims=['y', 'x', 'band'], 
                              coords={'y': np.arange(Predictions.shape[0]),
                                      'x': np.arange(Predictions.shape[1]), 
                                      'band': np.arange(Predictions.shape[2])})
plot_Predictions = Predictions_xr.hvplot.rgb(**general_settings, bands='band', title='Predictions')

#Classification
Classification = np.where(y_score>0.4, 1, 0)
#convert to 'xarray.DataArray'
Classification_xr = xr.DataArray(Classification, dims=['y', 'x'], 
                                 coords={'y': np.arange(Classification.shape[0]),
                                         'x': np.arange(Classification.shape[1])})
plot_Classification = Classification_xr.hvplot(**general_settings, cmap='viridis', colorbar=False, title='Classification')

In [ ]:
cplot =  plot_RGB + hv.Empty() + plot_FDI + plot_NDVI + plot_Predictions + plot_Classification
cplot.cols(2)

## Use Cases
In this section we validate our model on specific use cases. The images tested are provided in the [Zenodo repository](https://zenodo.org/record/5827377#.YdgfjGjMK9I).

### Plastic Litter Project (PLP) 2021
Here we validate the selected model on a Sentinel-2 scene capturing the deployed targets of the latest edition of the [Plastic Litter Project (PLP)](http://plp.aegean.gr/category/experiment-log-2021/) in Mytilene, Greece. The scene was acquired on the 21st of June 2021 (http://plp.aegean.gr/2021/06/21/target-2-placement-2/) and captures two targets:
- a circular 28m diameter target composed of high-density polyethylene (HDPE) mesh, covering a total area of 615m$^2$;
- a wooden target built with a rectangular grid pattern achieving the same pixel area coverage as the HDPE mesh target.

In [ ]:
def visualise(**images):
    '''Visualisation function'''
    n = len(images)
    plt.figure(figsize=(25, 25))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(name)
        plt.imshow(image)
    plt.show()

def run_preds(test_image):
    '''Run the model over the test image and plot the results'''
    with rio.open(test_image, "r") as src_pred:
        meta = src_pred.meta
        image = src_pred.read()

    #If L1C image (13 bands), read only the 12 bands compatible with L2A data
    if (image.shape[0] == 13):
        image = image[[l1cbands.index(b) for b in l2abands]]

    #Compute RGB representation
    tensor = np.stack([image[l2abands.index('B4')], image[l2abands.index('B3')], image[l2abands.index('B2')]])
    rgb = equalize_hist(tensor.swapaxes(0,1).swapaxes(1,2))

    image = image.astype(float)
    image *= 1e-4
    image = torch.from_numpy(image)

    #Compute predictions
    with torch.no_grad():
        x = image.unsqueeze(0)
        y_logits = torch.sigmoid(model(x.to(device)).squeeze(0))
        y_score = y_logits.cpu().detach().numpy()[0]

    #Compute the NDVI and FDI bands corresponding to the image
    fdi = np.squeeze(np.expand_dims(calculate_fdi(image),0),0)
    ndvi = calculate_ndvi(image)
    
    visualise(
        RGB = rgb,
        FDI = cmap_magma(norm_fdi(fdi)),
        NDVI = cmap_viridis(norm_ndvi(ndvi)),
        Predictions = cmap_magma(norm(y_score)),
        Classification = np.where(y_score>0.5, 1, 0)
    )

We can see that the model was able to accurately predict floating-object scores.

In [ ]:
#Read test image
test_image = './ocean-modelling-litter-philab/mytilini_20210621.tif'
run_preds(test_image)

### Sargassum
Here we validate our model in a Sentinel-2 scene captured on the 14th of September 2018, likely to contain sargassum in the coastal area of Cancun, Mexico.

In [ ]:
#Read test image
test_image = './ocean-modelling-litter-philab/cancun_20180914.tif'
run_preds(test_image)

### Ice
Here we validate our model in a Sentinel-2 scene captured on the 30th of January 2018 containing ice in the coastal area of Tangshan, China.

In [ ]:
#Read test image
test_image = './ocean-modelling-litter-philab/tangshan_20180130.tif'
run_preds(test_image)

## Summary

In this notebook we have explored the use of deep learning-based segmentation models to detect and delineate floating targets on Sentinel-2 coastal scenes. We have developed the following pipeline:


* using the `sentinelsat` package, fetch a Sentinel-2 image over a coastal area from the Sentinel-Hub service, specifying the bounding box coordinates describing an Area Of Interest (AOI) in the reference system WGS84 (EPSG:4326);
* re-project and clip the retrieved Sentinel-2 image to the exact AOI, using the `geopandas` library;
* visualise the different Sentinel-2 bands on an interactive map, using the `hvplot` package;
* display the AOI on a real map;
* export the Sentinel-2 scene as a .tif image, using the `rasterio` library;
* provide an option to fetch data from a Zenodo repository, using the `zenodo_get` package;
* compute model predictions on several use cases using pre-trained model weights for U-Net;
* visualise the RGB image, the NDVI and FDI indices along with the model predictions on interactive maps.